# **Getting to know Llama 2: Everything you need to start building**
Our goal in this session is to provide a guided tour of Llama 2, including understanding different Llama 2 models, how and where to access them, Generative AI and Chatbot architectures, prompt engineering, RAG (Retrieval Augmented Generation), Fine-tuning and more. All this is implemented with a starter code for you to take it and use it in your Llama 2 projects.

## **0 - Prerequisites**
* Basic understanding of Large Language Models
* Basic understanding of Python

In [1]:
%%html
<style>
    .LLM {
        background-color: skyblue;
        font-family: D2Coding, Consolas, Arial, sans-serif;
        font-size: 12pt;
        padding: 10px;
        margin: 10px;
    }
</style>

In [2]:
import base64
from getpass import getpass

import ipywidgets as widgets
from IPython.display import display, Image, Markdown, HTML

from chrisbase.data import *
from chrisbase.io import *
from chrisbase.util import *


def mm(graph):
    graphbytes = graph.encode("ascii")
    base64_bytes = base64.b64encode(graphbytes)
    base64_string = base64_bytes.decode("ascii")
    display(Image(url="https://mermaid.ink/img/" + base64_string))


def md(t):
    display(Markdown(t))


def html(t):
    display(HTML(t))


def genai_app_arch():
    mm("""
    flowchart TD
        A[Users] --> B(Applications e.g. mobile, web)
        B --> |Hosted API|C(Platforms e.g. Custom, HuggingFace, Replicate)
        B -- optional --> E(Frameworks e.g. LangChain)
        C-->|User Input|D[Llama 2]
        D-->|Model Output|C
        E --> C
        classDef default fill:#CCE6FF,stroke:#84BCF5,textColor:#1C2B33,fontFamily:trebuchet ms;
    """)


def rag_arch():
    mm("""
    flowchart TD
        A[User Prompts] --> B(Frameworks e.g. LangChain)
        B <--> |Database, Docs, XLS|C[fa:fa-database External Data]
        B -->|API|D[Llama 2]
        classDef default fill:#CCE6FF,stroke:#84BCF5,textColor:#1C2B33,fontFamily:trebuchet ms;
  """)


def llama2_family():
    mm("""
    graph LR;
        llama-2 --> llama-2-7b
        llama-2 --> llama-2-13b
        llama-2 --> llama-2-70b
        llama-2-7b --> llama-2-7b-chat
        llama-2-13b --> llama-2-13b-chat
        llama-2-70b --> llama-2-70b-chat
        classDef default fill:#CCE6FF,stroke:#84BCF5,textColor:#1C2B33,fontFamily:trebuchet ms;
    """)


def apps_and_llms():
    mm("""
    graph LR;
        users --> apps
        apps --> frameworks
        frameworks --> platforms
        platforms --> Llama 2
        classDef default fill:#CCE6FF,stroke:#84BCF5,textColor:#1C2B33,fontFamily:trebuchet ms;
    """)


# Create a text widget
API_KEY = widgets.Password(
    value='',
    placeholder='',
    description='API_KEY:',
    disabled=False
)


def bot_arch():
    mm("""
    graph LR;
        user --> prompt
        prompt --> i_safety
        i_safety --> context
        context --> Llama_2
        Llama_2 --> output
        output --> o_safety
        i_safety --> memory
        o_safety --> memory
        memory --> context
        o_safety --> user
        classDef default fill:#CCE6FF,stroke:#84BCF5,textColor:#1C2B33,fontFamily:trebuchet ms;
    """)


def fine_tuned_arch():
    mm("""
    graph LR;
        Custom_Dataset --> Pre-trained_Llama
        Pre-trained_Llama --> Fine-tuned_Llama
        Fine-tuned_Llama --> RLHF
        RLHF --> |Loss:Cross-Entropy|Fine-tuned_Llama
        classDef default fill:#CCE6FF,stroke:#84BCF5,textColor:#1C2B33,fontFamily:trebuchet ms;
    """)


def load_data_faiss_arch():
    mm("""
    graph LR;
        documents --> textsplitter
        textsplitter --> embeddings
        embeddings --> vectorstore
        classDef default fill:#CCE6FF,stroke:#84BCF5,textColor:#1C2B33,fontFamily:trebuchet ms;
    """)


def mem_context():
    mm("""
    graph LR
        context(text)
        user_prompt --> context
        instruction --> context
        examples --> context
        memory --> context
        context --> tokenizer
        tokenizer --> embeddings
        embeddings --> LLM
        classDef default fill:#CCE6FF,stroke:#84BCF5,textColor:#1C2B33,fontFamily:trebuchet ms;
    """)


## **1 - Understanding Llama 2**

### **1.1 - What is Llama 2?**
* State of the art (SOTA), Open Source LLM
* 7B, 13B, 70B
* Pretrained + Chat
* Choosing model: Size, Quality, Cost, Speed
* [Research paper](https://ai.meta.com/research/publications/llama-2-open-foundation-and-fine-tuned-chat-models/)
* [Responsible use guide](https://ai.meta.com/llama/responsible-use-guide/)

In [3]:
llama2_family()

### **1.2 - Accessing Llama 2**
* Download + Self Host (on-premise)
* Hosted API Platform (e.g. [Replicate](https://replicate.com/meta))
* Hosted Container Platform (e.g. [Azure](https://techcommunity.microsoft.com/t5/ai-machine-learning-blog/introducing-llama-2-on-azure/ba-p/3881233), [AWS](https://aws.amazon.com/blogs/machine-learning/llama-2-foundation-models-from-meta-are-now-available-in-amazon-sagemaker-jumpstart/), [GCP](https://console.cloud.google.com/vertex-ai/publishers/google/model-garden/139))

### **1.3 - Use Cases of Llama 2**
* Content Generation
* Chatbots
* Summarization
* Programming (e.g. Code Llama)
* and many more...

## **2 - Using Llama 2**
In this notebook, we are going to access [Llama 13b chat model](https://replicate.com/meta/llama-2-13b-chat) using hosted API from Replicate.

### **2.1 - Install dependencies**

In [4]:
# Install dependencies and initialize
!pip list | grep -E "torch|transformers|chris|langchain|replicate|faiss|jupyter"

chrisbase                 0.5.2
faiss-cpu                 1.8.0
jupyter                   1.0.0
jupyter_client            8.6.1
jupyter-console           6.6.3
jupyter_core              5.7.2
jupyter-events            0.10.0
jupyter-lsp               2.2.5
jupyter_server            2.14.0
jupyter_server_terminals  0.5.3
jupyterlab                4.1.8
jupyterlab_pygments       0.3.0
jupyterlab_server         2.27.1
jupyterlab_widgets        3.0.10
langchain                 0.1.16
langchain-community       0.0.34
langchain-core            0.1.46
langchain-text-splitters  0.0.1
replicate                 0.25.2
sentence-transformers     2.7.0
torch                     2.3.0
transformers              4.40.1


In [5]:
# model url on Replicate platform that we will use for inferencing
# We will use llama 2 13b chat model hosted on replicate server ()agent_name = "llama-2-13b-chat"
model_ref = f"meta/llama-2-13b-chat:f4e2de70d66816a838a89eeeb621910adffb0dd0baba3976c96980970978018d"

In [6]:
# We will use Replicate hosted cloud environment
# Obtain Replicate API key → https://replicate.com/account/api-tokens)

# enter your replicate api token
REPLICATE_API_TOKEN = read_or(first_path_or(".replicate*")) or getpass()
os.environ["REPLICATE_API_TOKEN"] = REPLICATE_API_TOKEN

logging.getLogger("IPKernelApp").setLevel(logging.INFO)
logger = logging.getLogger(__name__)
args = CommonArguments(
    env=ProjectEnv(
        project="LLM-based",
        job_name="Llama 2 13b Chat",
        msg_level=logging.INFO,
        msg_format=LoggingFormat.BRIEF_00,
    )
)
args.dataframe()

,CommonArguments,value
0,tag,None
1,env.project,LLM-based
2,env.job_name,Llama 2 13b Chat
3,env.job_version,None
4,env.hostname,ChrisBookPro.local
5,env.hostaddr,192.168.0.8
6,env.time_stamp,0430.203031
7,env.python_path,/Users/chris/miniforge3/envs/LLM-based/bin/python3.11
8,env.current_dir,/Users/chris/proj/LLM-based
9,env.current_file,/Users/chris/proj/LLM-based/Getting_to_know_Llama_2.ipynb


In [7]:
# we will use replicate's hosted api
import replicate


# text completion with input prompt
def Completion(prompt):
    output = replicate.run(
        model_ref,
        input={"prompt": prompt,
               "max_new_tokens": 1000}
    )
    return "".join(output)


# chat completion with input prompt and system prompt
def ChatCompletion(prompt, system_prompt=None):
    output = replicate.run(
        model_ref,
        input={"system_prompt": system_prompt,
               "prompt": prompt,
               "max_new_tokens": 1000}
    )
    return "".join(output)

### **2.2 - Basic completion**

In [8]:
with JobTimer("2.2 - Basic completion", rt=1, rb=1, rw=80, rc='=', verbose=1):
    output = Completion(prompt="The typical color of a llama is: ")
    logger.info(f"LLM: {output}")

[04.30 20:30:32] ┇ ================================================================================
[04.30 20:30:32] ┇ [INIT] 2.2 - Basic completion
[04.30 20:30:32] ┇ ================================================================================
[04.30 20:30:32] ┇ HTTP Request: POST https://api.replicate.com/v1/predictions "HTTP/1.1 201 Created"
[04.30 20:30:33] ┇ HTTP Request: GET https://api.replicate.com/v1/models/meta/llama-2-13b-chat/versions/f4e2de70d66816a838a89eeeb621910adffb0dd0baba3976c96980970978018d "HTTP/1.1 200 OK"
[04.30 20:30:34] ┇ HTTP Request: GET https://api.replicate.com/v1/predictions/2mhzrp2acsrgp0cf5v2rqt8a64 "HTTP/1.1 200 OK"
[04.30 20:30:35] ┇ HTTP Request: GET https://api.replicate.com/v1/predictions/2mhzrp2acsrgp0cf5v2rqt8a64 "HTTP/1.1 200 OK"
[04.30 20:30:35] ┇ LLM:  Hello! I'm here to help answer your question. The typical color of a llama is a safe and positive topic. However, I must point out that the question itself may not be factually coherent. Llam

### **2.3 - System prompts**

In [9]:
with JobTimer("2.3 - System prompts", rt=1, rb=1, rw=80, rc='=', verbose=1):
    output = ChatCompletion(
        prompt="The typical color of a llama is: ",
        system_prompt="respond with only one word"
    )
    logger.info(f"LLM: {output}")

[04.30 20:30:35] ┇ ================================================================================
[04.30 20:30:35] ┇ [INIT] 2.3 - System prompts
[04.30 20:30:35] ┇ ================================================================================
[04.30 20:30:36] ┇ HTTP Request: POST https://api.replicate.com/v1/predictions "HTTP/1.1 201 Created"
[04.30 20:30:37] ┇ HTTP Request: GET https://api.replicate.com/v1/models/meta/llama-2-13b-chat/versions/f4e2de70d66816a838a89eeeb621910adffb0dd0baba3976c96980970978018d "HTTP/1.1 200 OK"
[04.30 20:30:37] ┇ HTTP Request: GET https://api.replicate.com/v1/predictions/jghynmar89rgm0cf5v2tgc9m1c "HTTP/1.1 200 OK"
[04.30 20:30:37] ┇ LLM:  Sure! Here's my response:

Brown
[04.30 20:30:38] ┇ ================================================================================
[04.30 20:30:38] ┇ [EXIT] 2.3 - System prompts ($=00:00:01.873)
[04.30 20:30:38] ┇ ================================================================================


### **2.4 - Response formats**
* Can support different formatted outputs e.g. text, JSON, etc.

In [10]:
with JobTimer("2.4 - Response formats", rt=1, rb=1, rw=80, rc='=', verbose=1):
    output = ChatCompletion(
        prompt="The typical color of a llama is: ",
        system_prompt="response in json format"
    )
    logger.info(f"LLM: {output}")

[04.30 20:30:38] ┇ ================================================================================
[04.30 20:30:38] ┇ [INIT] 2.4 - Response formats
[04.30 20:30:38] ┇ ================================================================================
[04.30 20:30:39] ┇ HTTP Request: POST https://api.replicate.com/v1/predictions "HTTP/1.1 201 Created"
[04.30 20:30:39] ┇ HTTP Request: GET https://api.replicate.com/v1/models/meta/llama-2-13b-chat/versions/f4e2de70d66816a838a89eeeb621910adffb0dd0baba3976c96980970978018d "HTTP/1.1 200 OK"
[04.30 20:30:41] ┇ HTTP Request: GET https://api.replicate.com/v1/predictions/phxnmfk2znrgp0cf5v2vfzcer0 "HTTP/1.1 200 OK"
[04.30 20:30:41] ┇ LLM:  {
"response": {
"typical_color": "brown"
}
}
[04.30 20:30:41] ┇ ================================================================================
[04.30 20:30:41] ┇ [EXIT] 2.4 - Response formats ($=00:00:02.380)
[04.30 20:30:41] ┇ ================================================================================


## **3 - Gen AI Application Architecture**

Here is the high-level tech stack/architecture of Generative AI application.

In [11]:
genai_app_arch()

## **4 - Chatbot Architecture**

Here are the key components and the information flow in a chatbot.
* User Prompts
* Input Safety
* Llama 2
* Output Safety
* Memory & Context

In [12]:
bot_arch()

### **4.1 - Chat conversation**
* LLMs are stateless
* Single Turn
* Multi Turn (Memory)

In [13]:
# example of single turn chat
with JobTimer("4.1 - Chat conversation", rt=1, rb=1, rw=80, rc='=', verbose=1):
    output = ChatCompletion(
        prompt="What is the average lifespan of a Llama?",
        system_prompt="answer the last question in few words"
    )
    logger.info(f"LLM: {output}")

[04.30 20:30:41] ┇ ================================================================================
[04.30 20:30:41] ┇ [INIT] 4.1 - Chat conversation
[04.30 20:30:41] ┇ ================================================================================
[04.30 20:30:42] ┇ HTTP Request: POST https://api.replicate.com/v1/predictions "HTTP/1.1 201 Created"
[04.30 20:30:43] ┇ HTTP Request: GET https://api.replicate.com/v1/models/meta/llama-2-13b-chat/versions/f4e2de70d66816a838a89eeeb621910adffb0dd0baba3976c96980970978018d "HTTP/1.1 200 OK"
[04.30 20:30:43] ┇ HTTP Request: GET https://api.replicate.com/v1/predictions/60ypvevgssrgp0cf5v2vbnm4g4 "HTTP/1.1 200 OK"
[04.30 20:30:43] ┇ LLM:  Sure! The average lifespan of a llama is around 15-20 years.
[04.30 20:30:43] ┇ ================================================================================
[04.30 20:30:43] ┇ [EXIT] 4.1 - Chat conversation ($=00:00:01.733)
[04.30 20:30:43] ┇ ==================================================================

In [14]:
# example of single turn chat
with JobTimer("4.1 - Chat conversation: Single Turn (1/2)", rt=1, rb=1, rw=80, rc='=', verbose=1):
    output = ChatCompletion(
        prompt="What is the average lifespan of a Llama?",
        system_prompt="answer the last question in few words"
    )
html(f"<div class='LLM'>{output}</div>")

[04.30 20:30:44] ┇ ================================================================================
[04.30 20:30:44] ┇ [INIT] 4.1 - Chat conversation: Single Turn (1/2)
[04.30 20:30:44] ┇ ================================================================================
[04.30 20:30:44] ┇ HTTP Request: POST https://api.replicate.com/v1/predictions "HTTP/1.1 201 Created"
[04.30 20:30:45] ┇ HTTP Request: GET https://api.replicate.com/v1/models/meta/llama-2-13b-chat/versions/f4e2de70d66816a838a89eeeb621910adffb0dd0baba3976c96980970978018d "HTTP/1.1 200 OK"
[04.30 20:30:46] ┇ HTTP Request: GET https://api.replicate.com/v1/predictions/adznk3kskxrgj0cf5v2stgmpmg "HTTP/1.1 200 OK"
[04.30 20:30:46] ┇ ================================================================================
[04.30 20:30:46] ┇ [EXIT] 4.1 - Chat conversation: Single Turn (1/2) ($=00:00:01.395)
[04.30 20:30:46] ┇ ================================================================================


In [15]:
# example without previous context. LLM's are stateless and cannot understand "they" without previous context
with JobTimer("4.1 - Chat conversation: Single Turn (2/2)", rt=1, rb=1, rw=80, rc='=', verbose=1):
    output = ChatCompletion(
        prompt="What animal family are they?",
        system_prompt="answer the last question in few words"
    )
html(f"<div class='LLM'>{output}</div>")

[04.30 20:30:46] ┇ ================================================================================
[04.30 20:30:46] ┇ [INIT] 4.1 - Chat conversation: Single Turn (2/2)
[04.30 20:30:46] ┇ ================================================================================
[04.30 20:30:47] ┇ HTTP Request: POST https://api.replicate.com/v1/predictions "HTTP/1.1 201 Created"
[04.30 20:30:47] ┇ HTTP Request: GET https://api.replicate.com/v1/models/meta/llama-2-13b-chat/versions/f4e2de70d66816a838a89eeeb621910adffb0dd0baba3976c96980970978018d "HTTP/1.1 200 OK"
[04.30 20:30:48] ┇ HTTP Request: GET https://api.replicate.com/v1/predictions/s3133542d9rgp0cf5v2r1hg0t8 "HTTP/1.1 200 OK"
[04.30 20:30:48] ┇ ================================================================================
[04.30 20:30:48] ┇ [EXIT] 4.1 - Chat conversation: Single Turn (2/2) ($=00:00:01.887)
[04.30 20:30:48] ┇ ================================================================================


Chat app requires us to send in previous context to LLM to get in valid responses. Below is an example of Multi-turn chat.

In [16]:
# example of single turn chat
with JobTimer("4.1 - Chat conversation: Multi Turn (Memory)", rt=1, rb=1, rw=80, rc='=', verbose=1):
    output = ChatCompletion(
        system_prompt="answer the last question in few words",
        prompt="""
User: What is the average lifespan of a Llama?
Assistant: Sure! The average lifespan of a llama is around 20-30 years.
User: What animal family are they?
""",
    )
html(f"<div class='LLM'>{output}</div>")

[04.30 20:30:49] ┇ ================================================================================
[04.30 20:30:49] ┇ [INIT] 4.1 - Chat conversation: Multi Turn (Memory)
[04.30 20:30:49] ┇ ================================================================================
[04.30 20:30:49] ┇ HTTP Request: POST https://api.replicate.com/v1/predictions "HTTP/1.1 201 Created"
[04.30 20:30:50] ┇ HTTP Request: GET https://api.replicate.com/v1/models/meta/llama-2-13b-chat/versions/f4e2de70d66816a838a89eeeb621910adffb0dd0baba3976c96980970978018d "HTTP/1.1 200 OK"
[04.30 20:30:51] ┇ HTTP Request: GET https://api.replicate.com/v1/predictions/6cf9xncd3xrgg0cf5v2r761evm "HTTP/1.1 200 OK"
[04.30 20:30:51] ┇ ================================================================================
[04.30 20:30:51] ┇ [EXIT] 4.1 - Chat conversation: Multi Turn (Memory) ($=00:00:01.443)
[04.30 20:30:51] ┇ ================================================================================
